In [ ]:
import boto3

vpc_client = boto3.client('ec2')

In [ ]:
# Note that is you run this in an older account that supports EC2-Classic there might not be 
# a default VPC
response = vpc_client.describe_vpcs()
vpcs = response['Vpcs']
print vpcs

def_vpc = ''
for vpc in vpcs:
    if vpc['IsDefault'] == True:
        def_vpc = vpc['VpcId']
        break
        
print 'default vpc is {}'.format(def_vpc)

In [ ]:
response = vpc_client.describe_subnets(
    Filters=[
        {
            'Name':'vpc-id',
            'Values': [
                def_vpc
            ]
        }
    ]
)

print response

subnets = response['Subnets']

azs = []

for sn in subnets:
    azs.append(sn['AvailabilityZone'])


az_list = ",".join(azs)
print az_list

In [ ]:
# Things to consider when creating a memcached cluster
# a. There is no replication so when a node is lost all the data is lost.
#    More nodes in a cluster means less data is lost when a node crashes
# b. Spreading nodes across availability zones reduces the amount of data
#    lost during an availability zone outage.

In [ ]:
import boto3

client = boto3.client('elasticache')

In [ ]:
response = client.create_cache_cluster(
    CacheClusterId='memcached-sample-cluster',
    AZMode='cross-az',
    ### Continue
    etc
)